# 即刻执行Eager execution


TensorFlow的即刻执行是一个命令式编程环境，它不构建计算图而立刻进行计算，操作返回具体值而不是构建后面运行的计算图。这使得TensorFlow'较容易上手和调试模型，也减低了模板代码的需要。为了学习本内容，你可以同构一个交互Python解释器运行例子代码。

即刻执行是一个灵活的机器学习研究和实验平台，提供：

* *符合直觉的接口*—自然的组织你的代码，使用Python数据结构，快速的在小模型和小数据上迭代。.
* *容易调试*—直接调用操作来检查模型运行和测试修改。使用标准Python调式工具进行立即错误报告。.
* *自然的控制流*—使用Python控制流而不是图控制流，简化动态模型的描述。

即刻自行支持绝大多数TensorFlow操作和GPU加速。

注意：有些模型使用即刻执行可能面临额外负担，性能提高正在进行中。

## 设置和基本用法

In [ ]:
import os

import tensorflow as tf

import cProfile

在TensorFlow2中，即刻执行缺省是激活的。

In [ ]:
tf.executing_eagerly()

现在你可以执行TensorFlow操作，并立刻得到结果:

In [ ]:
x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))

运行即刻执行改变了TensorFlow操作的行为--现在它们立即执行并返回结果，`tf.Tensor`对象引用具体的值，而不再是计算图中节点的符号句柄。由于没有构建的计算图和在一个会话中运行它，容易使用`print()` 或调试器查看结果。计算、打印和检查张量的值不需要打断梯度计算流程。

即刻执行和[NumPy](http://www.numpy.org/)一起工作. NumPy操作接受 `tf.Tensor` 参数. TensorFlow
`tf.math` 操作将Python对象和NumPy数组转换成 `tf.Tensor` 对象. 
`tf.Tensor.numpy`方法将对象值作为NumPy `ndarray`返回。

In [ ]:
a = tf.constant([[1, 2],
                 [3, 4]])
print(a)

In [ ]:
# 广播支持
b = tf.add(a, 1)
print(b)

In [ ]:
# 支持运算符重载
print(a * b)

In [ ]:
# 和numpy一起工作
import numpy as np

c = np.multiply(a, b)
print(c)

In [ ]:
# 从张量获得numpy值:
print(a.numpy())
# => [[1 2]
#     [3 4]]

## 动态控制流

即刻执行的一个最大的好处是宿主语言的功能在模型执行中可用。因此，例如，比较容易写[fizzbuzz](https://en.wikipedia.org/wiki/Fizz_buzz):

In [ ]:
def fizzbuzz(max_num):
  counter = tf.constant(0)
  max_num = tf.convert_to_tensor(max_num)  # python对象转换成张量
  for num in range(1, max_num.numpy()+1):  # 张量用在python结构中
    num = tf.constant(num)
    if int(num % 3) == 0 and int(num % 5) == 0:
      print('FizzBuzz')
    elif int(num % 3) == 0:
      print('Fizz')
    elif int(num % 5) == 0:
      print('Buzz')
    else:
      print(num.numpy())
    counter += 1

In [ ]:
fizzbuzz(15)

条件依赖张量值，并在运行时打印这些值。

## 即刻训练

### 梯度计算

[自动求导](https://en.wikipedia.org/wiki/Automatic_differentiation)对于实现机器学习算法，例如用于训练神经网络的[backpropagation](https://en.wikipedia.org/wiki/Backpropagation) 非常有用。在即刻执行中，使用`tf.GradientTape`跟踪梯度计算的操作。

使用`tf.GradientTape` 来训练或即刻计算梯度，这对于复杂的训练循环尤其有用。

由于每次调用会产生不同的操作，所有前向操作记录到tape。为了计算梯度，反向play tape，然后丢弃。一个 `tf.GradientTape`只能计算一个梯度，后续调用会抛出运行时错误。

In [ ]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
  loss = w * w  # 记录前向操作

grad = tape.gradient(loss, w)  # 求导
print(grad)  # => tf.Tensor([[ 2.]], shape=(1, 1), dtype=float32)

### 训练模型

下面例子创建一个多层模型，对标准的MNIST手写数字进行分类。它展示了在即刻执行环境中使用优化器和层API来构建可训练的图。

In [ ]:
# 获得和格式化mnist数据
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

dataset = tf.data.Dataset.from_tensor_slices(
  (tf.cast(mnist_images[...,tf.newaxis]/255, tf.float32),
   tf.cast(mnist_labels,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

In [ ]:
# 构建模型
mnist_model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(16,[3,3], activation='relu',
                         input_shape=(None, None, 1)),  # 长和宽为什么不需要指定？
  tf.keras.layers.Conv2D(16,[3,3], activation='relu'),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10)
])

即使不训练，即刻执行也可以调用模型和查看输出：

In [ ]:
for images,labels in dataset.take(1):
  print("Logits: ", mnist_model(images[0:1]).numpy())

尽管keras模型有一个内置的训练循环(使用 `fit` 方法), 有时还是需要定制。这里的例子解释用即刻执行实现训练循环：

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

loss_history = []

注意：使用`tf.debugging` 检查条件是否成立，这在即刻执行和图执行没有问题。

In [ ]:
def train_step(images, labels):
  with tf.GradientTape() as tape:
    logits = mnist_model(images, training=True)  # 前向执行模型
    
    # 添加检查输出形状的断言.
    tf.debugging.assert_equal(logits.shape, (32, 10))
    
    loss_value = loss_object(labels, logits)  # 计算损失

  loss_history.append(loss_value.numpy().mean())
  grads = tape.gradient(loss_value, mnist_model.trainable_variables)  # 求梯度
  optimizer.apply_gradients(zip(grads, mnist_model.trainable_variables))  # 反向传播更新

In [ ]:
def train(epochs):
  for epoch in range(epochs):
    for (batch, (images, labels)) in enumerate(dataset):
      train_step(images, labels)
    print ('Epoch {} finished'.format(epoch))

In [ ]:
train(epochs = 3)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(loss_history)
plt.xlabel('Batch #')
plt.ylabel('Loss [entropy]')

### 变量和优化器

`tf.Variable`对象存储可变的  `tf.Tensor`-像训练期间访问的值，使得自动求导比较容易。 

变量的集合以及操作变量的方法可以包装到层或模型中，参考 [定制Keras层和模型] 了解更多细节. Keras层和模型的主要区别是，模型有  `Model.fit`, `Model.evaluate`, 和 `Model.save`这样的方法。

例如，前面的自动求导例子可以重写如下：

In [ ]:
class Linear(tf.keras.Model):
  def __init__(self):
    super(Linear, self).__init__()
    self.W = tf.Variable(5., name='weight')
    self.B = tf.Variable(10., name='bias')
  def call(self, inputs):
    return inputs * self.W + self.B

In [ ]:
# 3 * x + 2附件的玩具数据集
NUM_EXAMPLES = 2000
training_inputs = tf.random.normal([NUM_EXAMPLES])
noise = tf.random.normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

# 需要优化的损失函数
def loss(model, inputs, targets):
  error = model(inputs) - targets
  return tf.reduce_mean(tf.square(error))

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, [model.W, model.B])

接下来:

1. 创建模型；
2. 求损失函数相对于模型参数的导数；
3. 采用某种策略，基于导数更新变量或模型的参数

In [ ]:
model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

print("Initial loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))

steps = 300
for i in range(steps):
  grads = grad(model, training_inputs, training_outputs)
  optimizer.apply_gradients(zip(grads, [model.W, model.B]))
  if i % 20 == 0:
    print("Loss at step {:03d}: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))

In [ ]:
print("Final loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))

In [ ]:
print("W = {}, B = {}".format(model.W.numpy(), model.B.numpy()))

注意：变量直到最后一个python对象引用删除和变量删除才persist。

### 基于对象的保存


`tf.keras.Model`包含一个方便的方法 `save_weights`， 它可以容易的创建一个检查点：

In [ ]:
model.save_weights('weights')
status = model.load_weights('weights')

使用`tf.train.Checkpoint`，你可以对这个过程进行完全的控制。

这一部分是 [训练检查点指南]的简化版本.


In [ ]:
x = tf.Variable(10.)
checkpoint = tf.train.Checkpoint(x=x)

In [ ]:
x.assign(2.)   # 给变量赋新值，并保存.
checkpoint_path = './ckpt/'
checkpoint.save('./ckpt/')

In [ ]:
x.assign(11.)  # 保存后改变变量值.

# 从检测点恢复变量
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path))

print(x)  # => 2.0

为了保存和装载模型, `tf.train.Checkpoint`保存对象的内部状态，不需要隐藏变量。为了记录模型、优化器和全局步的状态，将它们传递给`tf.train.Checkpoint`:

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(16,[3,3], activation='relu'),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

checkpoint_dir = 'path/to/model_dir'
if not os.path.exists(checkpoint_dir):
  os.makedirs(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

root = tf.train.Checkpoint(optimizer=optimizer,
                           model=model)

root.save(checkpoint_prefix)
root.restore(tf.train.latest_checkpoint(checkpoint_dir))

注意：在许多训练循环中，`tf.train.Checkpoint.restore`方法调用后，变量被创建，这些变量在创建后立刻恢复，断言确保检查点完整装载。

### 面向对象的指标

`tf.keras.metrics`作为对象保存，使用新数据调用指标的可调用接口更新指标，使用`tf.keras.metrics.result`检索指标值，例如：

In [ ]:
m = tf.keras.metrics.Mean("loss")
m(0)
m(5)
m.result()  # => 2.5
m([8, 9])
m.result()  # => 5.5

### 摘要和TensorBoard

[TensorBoard]是一个理解、调试和优化模型训练过程的工具。它使用程序执行中写的摘要事件。 

你可以使用 `tf.summary` 来记录即刻执行中的变量的摘要，例如，每100训练步记录一次`loss`的摘要信息：

In [ ]:
logdir = "./tb/"
writer = tf.summary.create_file_writer(logdir)

steps = 1000
with writer.as_default():  # 或者在循环前调用writer.set_as_default().
  for i in range(steps):
    step = i + 1
    # 用真实训练函数计算损失.
    loss = 1 - 0.001 * step
    if step % 100 == 0:
      tf.summary.scalar('loss', loss, step=step)

In [ ]:
!dir tb/

## 高级自动求导主题

### 动态模型

`tf.GradientTape`也可以用在动态模型中。这是一个
[反向跟踪线性搜索backtracking line search](https://wikipedia.org/wiki/Backtracking_line_search)算法的例子，看起来像numpy代码，除了有梯度和可微外（尽管有复杂的控制流）:

In [ ]:
def line_search_step(fn, init_x, rate=1.0):
  with tf.GradientTape() as tape:
    # 变量自动跟踪，但是为了计算梯度，需要`watch` 它.
    tape.watch(init_x)
    value = fn(init_x)
  grad = tape.gradient(value, init_x)
  grad_norm = tf.reduce_sum(grad * grad)
  init_value = value
  while value > init_value - rate * grad_norm:
    x = init_x - rate * grad
    value = fn(x)
    rate /= 2.0
  return x, value

### 定制梯度

梯度定制是override梯度的简单方法，在前向函数中，定义相对于输入、输出和中间结果的梯度。例如，下面是在反向过程中clip梯度模的简单方法:

In [ ]:
@tf.custom_gradient
def clip_gradient_by_norm(x, norm):
  y = tf.identity(x)
  def grad_fn(dresult):
    return [tf.clip_by_norm(dresult, norm), None]
  return y, grad_fn

梯度定制常用来为一个操作序列提供数值上稳定的梯度:

In [ ]:
def log1pexp(x):
  return tf.math.log(1 + tf.exp(x))

def grad_log1pexp(x):
  with tf.GradientTape() as tape:
    tape.watch(x)
    value = log1pexp(x)
  return tape.gradient(value, x)


In [ ]:
# 在x=0处，梯度计算工作良好.
grad_log1pexp(tf.constant(0.)).numpy()

In [ ]:
# 但是, x = 100时因为数值不稳定，梯度计算失败.
grad_log1pexp(tf.constant(100.)).numpy()

`log1pexp`函数可以用梯度定制来简化。 下面实现复用 `tf.exp(x)`前向过程中计算的值，通过消除重复计算而变得更高效:

In [ ]:
@tf.custom_gradient
def log1pexp(x):
  e = tf.exp(x)
  def grad(dy):
    return dy * (1 - 1 / (1 + e))
  return tf.math.log(1 + e), grad

def grad_log1pexp(x):
  with tf.GradientTape() as tape:
    tape.watch(x)
    value = log1pexp(x)
  return tape.gradient(value, x)


In [ ]:
# 和前面一样，在x=0处，梯度计算工作良好.
grad_log1pexp(tf.constant(0.)).numpy()

In [ ]:
# 在x=100处，梯度计算同样没有问题
grad_log1pexp(tf.constant(100.)).numpy()

## 性能

在即刻执行中，计算自动装载到GPU，如果你想控制计算运行的位置，你可以将计算放到
`tf.device('/gpu:0')` 块中 (或对应的CPU设备):

In [ ]:
import time

def measure(x, steps):
  # TensorFlow initializes a GPU the first time it's used, exclude from timing.
  tf.matmul(x, x)
  start = time.time()
  for i in range(steps):
    x = tf.matmul(x, x)
  # tf.matmul can return before completing the matrix multiplication
  # (e.g., can return after enqueing the operation on a CUDA stream).
  # The x.numpy() call below will ensure that all enqueued operations
  # have completed (and will also copy the result to host memory,
  # so we're including a little more than just the matmul operation
  # time).
  _ = x.numpy()
  end = time.time()
  return end - start

shape = (1000, 1000)
steps = 200
print("Time to multiply a {} matrix by itself {} times:".format(shape, steps))

# Run on CPU:
with tf.device("/cpu:0"):
  print("CPU: {} secs".format(measure(tf.random.normal(shape), steps)))

# Run on GPU, if available:
if tf.config.experimental.list_physical_devices("GPU"):
  with tf.device("/gpu:0"):
    print("GPU: {} secs".format(measure(tf.random.normal(shape), steps)))
else:
  print("GPU: not found")

`tf.Tensor`对象可以拷贝到不同设备上执行它的操作:

In [ ]:
if tf.config.experimental.list_physical_devices("GPU"):
  x = tf.random.normal([10, 10])

  x_gpu0 = x.gpu()
  x_cpu = x.cpu()

  _ = tf.matmul(x_cpu, x_cpu)    # 运行在CPU
  _ = tf.matmul(x_gpu0, x_gpu0)  # 运行在GPU:0

### 基准

对于计算密集的模型，例如在GPU上训练
[ResNet50](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/eager/python/examples/resnet50)
，即刻执行性能和`tf.function` 执行性能相当。但是，对于计算较少的模型，性能差距变大，需要对有大量小操作的模型的热代码路径进行优化，以提高性能。

## 和函数一起工作

尽管即刻执行使得开发和调试更可交互，但是TensorFlow1.x的图执行对于分布式训练性能优化和产品部署有优点。为了弥补差距，TensorFlow2.0通过`tf.function`API 引入`function`，更多细节，参阅函数部分指南。